In [16]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Create a prompt template from the template string
template = "You are an artificial intelligence assistant, answer the question. {question}"
prompt = PromptTemplate.from_template(
    template=template
)

In [3]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an artificial intelligence assistant, answer the question. {question}')

In [4]:
#llm_chain = prompt | llm - This is what should have been give
llm_chain = prompt
question = "How does LangChain make LLM application development easier?"
print(llm_chain.invoke({"question": question}))

text='You are an artificial intelligence assistant, answer the question. How does LangChain make LLM application development easier?'


### Chat Prompt Template

To give system, human, ai categorization in the prompt

In [5]:
# Create a chat prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert that returns the colors present in a country's flag."),
        ("human", "France"),
        ("ai", "blue, white, red"),
        ("human", "{country}")
    ]
)

In [7]:
# Chain the prompt template and model, and invoke the chain
llm_chain = prompt_template

country = "Japan"
response = llm_chain.invoke({"country": country})
print(response)

messages=[SystemMessage(content="You are a geography expert that returns the colors present in a country's flag.", additional_kwargs={}, response_metadata={}), HumanMessage(content='France', additional_kwargs={}, response_metadata={}), AIMessage(content='blue, white, red', additional_kwargs={}, response_metadata={}), HumanMessage(content='Japan', additional_kwargs={}, response_metadata={})]


### FewShotPrompting

In [8]:
    # Create the examples list of dicts
examples = [
  {
    "question": "How many DataCamp courses has Jack completed?",
    "answer": "36"
  },
  {
    "question": "How much XP does Jack have on DataCamp?",
    "answer": "284,320XP"
  },
  {
    "question": "What technology does Jack learn about most on DataCamp?",
    "answer": "Python"
  }
]

In [10]:
# Complete the prompt for formatting answers
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

# Create the few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

prompt = prompt_template.invoke({"input": "What is Jack's favorite technology on DataCamp?"})
print(prompt.text)

Question: How many DataCamp courses has Jack completed?
36

Question: How much XP does Jack have on DataCamp?
284,320XP

Question: What technology does Jack learn about most on DataCamp?
Python

Question: What is Jack's favorite technology on DataCamp?


In [13]:
# Create an OpenAI chat LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key='<OPENAI_API_TOKEN>')

# Create and invoke the chain
llm_chain = prompt_template | llm
print(llm_chain.invoke({"input": "What is Jack's favorite technology on DataCamp?"}))

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <OPENAI_******KEN>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [14]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"], # This is given so that the thing in {} in the next line is understood as vatiable name. basically uses fstring in background
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "Play Golf"}))

text='I want to learn how to Play Golf. Can you suggest how I can learn this step-by-step?'


In [17]:
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a concise plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
# seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()} | time_prompt | llm | StrOutputParser())
seq_chain = ({"learning_plan": learning_prompt | StrOutputParser()} | time_prompt | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "Play Golf"}))

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=StringPromptValue(text='I...arn this step-by-step?'), input_type=StringPromptValue]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type